In [112]:
# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **  Considering the question it can be solved by making it a classification problem. As we are only forming two groups of the students which needs early intervention and students who don't. Also there are many ways to make it a regression problem too. like having multiple sets of students with some percent chance of intervention, by making it a quantized solution, this problem can be easily converted from classification to regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [86]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [87]:
#print(student_data.head())
#print(sum(student_data["passed"]=='yes'))
print(student_data[0:10])

  school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4   at_home   teacher   
1     GP   F   17       U     GT3       T     1     1   at_home     other   
2     GP   F   15       U     LE3       T     1     1   at_home     other   
3     GP   F   15       U     GT3       T     4     2    health  services   
4     GP   F   16       U     GT3       T     3     3     other     other   
5     GP   M   16       U     LE3       T     4     3  services     other   
6     GP   M   16       U     LE3       T     2     2     other     other   
7     GP   F   17       U     GT3       A     4     4     other   teacher   
8     GP   M   15       U     LE3       A     3     2  services     other   
9     GP   M   15       U     GT3       T     3     4     other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3   

In [88]:
# TODO: Calculate number of students
n_students = (len(student_data))

# TODO: Calculate number of features
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = sum(student_data["passed"]=='yes')

# TODO: Calculate failing students
n_failed = sum(student_data["passed"]=='no')

# TODO: Calculate graduation rate
grad_rate = n_passed*100./n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [89]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [90]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [91]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
percent= num_test/(num_test+num_train)

Xtrain,Xtest,Ytrain,Ytest=cross_validation.train_test_split(X_all,y_all,test_size=0.24, random_state=0)
# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = Xtrain
X_test = Xtest
y_train = Ytrain
y_test = Ytest

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** 1. Logistic Regression 2. Decision Trees 3. K- Nearest Neighbors

1- About Logistic Regression, I have seen them being used in Medical industry where it was being used to detect benign and malignant samples of body parts. In this example we can see the logistic regression being applied to the womens pelvic masses. https://www.ncbi.nlm.nih.gov/pubmed/11778996 Logistic Regression works best when there is just 1 decision boundary as in this example, benign and malignant samples were able to be seperated well using single decision boundary and thus is good to be used here. The model might fell for some really non-linear dataset, so even though it won't overfit it will be difficult to fit a very nonlinear dataset. It is only an assumption right now that the passed and failed students will be able to seperated from a single decision boundary thus I wanted to give this model a try.


2- Decision trees are good for complex datasets. Like in this data set, weather forecasting is done using decision trees which I think is really a suitable example for decision trees. http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.403.3726&rep=rep1&type=pdf The weather forecast is done by providing the set of very complex weather data to its decision vector pulling out the output of the forecast. Where Logistic regression fails to deal with these kind of datasets, decision trees fits really well with the non linearity, with added disadvantage of overfitting. But that can be overcome by keeping a moderate complexity of the tree. I am picking this model as I wanted to do the comparison between logistic regression and decision trees. As both models are independently good and even after visualising the dataset, I am very unsure if it can be solved with a simple decision boundary or the complex one. The result of this experiment will surely help us to decide which one is good for the model


3- K-Nearest Neighbors as I wanted to go for an instance based approach for the dataset. The students must have a trend where they fail/pass due to their some other tendancy. This trend can be observed by memorising the whole dataset and then approximating them using their nearest neighboring datapoints. In this example, http://www.sciencedirect.com/science/article/pii/S0960148108002358 The power produced by wind farms is being predicted using this K-NN model. This is really a good approach as you already have the information about the power from the reading and information about the weather. You can easily plug it to make it predict the power in future. Disadvantage of this model can be it requires a big memory to hold the data. It can be made fast depending on the search algorithm used. The K value is also supposed to be tuned well as lower K may overfit the data and higher K may make it go underfitted. But the model is good when you don't want any learning process to occur. But you need to be careful while assuming if you have enough data for this kind of model or not. Less data in some cases may make it entirely useless.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [92]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [111]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import linear_model
from sklearn import tree
from sklearn import neighbors

# TODO: Initialize the three models
clf_A = linear_model.LogisticRegression(random_state=0)
clf_B = tree.DecisionTreeClassifier(random_state=0)
clf_C = neighbors.KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train
y_train_300 = y_train

#print(len(X_train_200))
clf_A.fit(X_train_100,y_train_100)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print("")
clf_B.fit(X_train_100,y_train_100)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print("")
clf_C.fit(X_train_100,y_train_100)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)

print("")

clf_A.fit(X_train_200,y_train_200)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print("")
clf_B.fit(X_train_200,y_train_200)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print("")
clf_C.fit(X_train_200,y_train_200)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)

print("")

clf_A.fit(X_train_300,y_train_300)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print("")
clf_B.fit(X_train_300,y_train_300)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print("")
clf_C.fit(X_train_300,y_train_300)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7612.

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6942.

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7068.

Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0030 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8380.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7794.

Training a DecisionTreeClassifier usin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0020             |        0.0010            |       0.8571           |    0.7612             |
| 200               |        0.0030         |         0.0000            |      0.8380            |        0.7794         |
| 300               |           0.0060              |       0.0010       |     0.8381             |    0.7910      |

** Classifer 2 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0010             |        0.0010           |        1.0000      |        0.6942         |
| 200               |     0.0020             |       0.0000             |        1.0000          |      0.7132           |
| 300               |      0.0050             |       0.0010            |         1.0000         |     0.7167     |

** Classifer 3 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0010              |     0.0010             |       0.7972           |      0.7068           |
| 200               |     0.0010              |     0.0040             |      0.8571            |      0.7121           |
| 300               |     0.0010              |     0.0090             |      0.8722            |      0.7482           |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** The models as are untuned did provide average results. The training time required for Logistic Regression Model and Decision trees was in the range of 0.001 to 0.005 range which even though is small, is more than the training time for K- Nearest Neighbors classifier. Whereas their prediction was very fast which is almost 0.001 seconds which is very small compared to the prediction time for K-Nearest Neighbors. The time complexity is not really necessity for this application but if we were implementing this for some real time application it could have been an issue as everytime K-NN is going to search through its space for prediction. 

There was a gap in their F1 Scores for both training and test sets. As expected before the complexity in the Decision trees made it go overfit, but it means it can handle the non-linearities. Also even though the difference between the F1 scores of Train/test is less in the Logistic Regression model, it kinda fails to adapt for the linearity as seen from the training score. The K-Nearest Neighbor seems to be performing well as the Training size increased, but the difference between scores of train-test was still very high. This can be improved by providing more data to it as this much data is not sufficient to make it a perfect model.

It is thus considered to move ahead with the decision trees as the overfitting problem can be cured by tuning the parameters as this model seems to be dealing well with the non-linearities of this model. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** Decision Trees classifier works with the 'decide and operate' principle. Like in our mind for some scenarios we are programmed to perform in a way, 'if this thing happens, do this'. The decision trees are exactly the same thing but can be very much multilayered and really dense. There can be stages after stages where from the previous decision we move ahead with some other decision. This principal we are going to use to classify students with future failing grades and passing grades. The model can be trained using some conditions to seperate the students in the above two classes. And once it is adjusted in that manner, it will be ready for the prediction.

The model first goes through the types of students and their traits. Like if a student has failed, it tries to see his actions duringhis time in school and accordingly decides how a student gets a failing grade and similarly for the student who passed the course, the model tries to understand his acts and then implements a logic to separate these two categories. This process is called training of a model. Once trained, while predicting, it will start to separate the students in small small sub classes using conditions like if the student is being absent for most of his classes, there is a higher chance that he won't be clearing that class in that semester. Similarly if the student is having good grades so far, and is having good health, he will try to maintain his grades in that manner. This is how a Decision tree will classify the set of students into the given two classes

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [130]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

from sklearn import grid_search
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'presort':('true','false'),'splitter':('random', 'best'),'criterion':('gini', 'entropy'),'min_samples_split':[20,30], 'max_depth':[5, 35]}

# TODO: Initialize the classifier
clf = tree.DecisionTreeClassifier(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='no')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf,parameters,f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 0.8622.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8088.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final F1_score for the tuned model is about 0.8622 for Training set and 0.8088 for the Testing set. It is significantly improved from the earlier untuned result which was about 1.0000 for the training set and 0.7377 for the test set. And as I was afraid earlier about the fact that the complexity is good for the non-linearity in decision trees, it may overfit and this is exactly what has happened when it was untuned. After tuning its parameters in the perfect range, we obtain a sweet spot for the classifier which gives very good results for both training and test sets. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.